 # Food Classification Using Deep Neural Networks and Transfer Learning

# 1. Data Splitting 

Split the data into training, validation, and test sets. Justify your choice.

In [0]:
import numpy as np
import time
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torch.utils.data.sampler import SubsetRandomSampler
import torchvision.transforms as transforms

import matplotlib.pyplot as plt


In [0]:
#move to gpu if possible

device = torch.device("cpu")
if torch.cuda.is_available():
    device = torch.device("cuda")
print("using device", device)

using device cuda


import shutil
import os
import random
random.seed = 2019
split = [600,800,1000]

for food in ['falafel','apple_pie','donuts','french_fries','macarons','nachos','onion_rings','oysters','pizza', 'mussels']:
    
    #create new empty folders
    if not os.path.exists('train/' + food):
        os.makedirs('train/' + food)
    if not os.path.exists('val/' + food):
        os.makedirs('val/' + food)
    if not os.path.exists('test/' + food):
        os.makedirs('test/' + food)
    
    #make a list of image file names, shuffle them
    pictures = os.listdir("Food Chosen/"+ food)
    pictures.sort()
    random.shuffle(pictures)
    
    #copy images into folders
    for pic in pictures[0:600]:
        shutil.copy("Food Chosen/"+ food +"/"+ pic,"train/"+food+"/"+pic)
        print("copy","Food Chosen/"+ food +"/"+ pic,"to","train/"+food+"/"+pic)
        
    for pic in pictures[600:800]:
        shutil.copy("Food Chosen/"+ food +"/"+ pic,"val/"+food+"/"+pic)
        print("copy","Food Chosen/"+ food +"/"+ pic,"to","val/"+food+"/"+pic)
        
    for pic in pictures[800:1000]:
        shutil.copy("Food Chosen/"+ food +"/"+ pic,"test/"+food+"/"+pic)
        print("copy","Food Chosen/"+ food +"/"+ pic,"to","test/"+food+"/"+pic)

# Load Data

In [0]:
def get_data_loader(target_classes, batch_size):
    """ Returns the indices for datapoints in the dataset that
    belongs to the desired target classes, a subset of all possible classes.

    Args:
        dataset: Dataset object
        classes: A list of strings denoting the name of each class
        target_classes: A list of strings denoting the name of the desired
                        classes. Should be a subset of the argument 'classes'
    Returns:
        indices: list of indices that have labels corresponding to one of the
                 target classes
    """
    classes = ('falafel','apple_pie','donuts','french_fries','macarons','nachos','onion_rings','oysters','pizza', 'mussels')


    transform = transforms.Compose(
    [transforms.Resize((224,224), interpolation=2), transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])

    trainset = torchvision.datasets.ImageFolder(root = 'train',
                                                 transform=transform)
 
    train_loader = torch.utils.data.DataLoader(trainset, batch_size = batch_size,
                                              num_workers = 1, shuffle = True)

    
    val_set = torchvision.datasets.ImageFolder(root='val',
                                                 transform=transform)
    val_loader = torch.utils.data.DataLoader(val_set, batch_size=batch_size,
                                              num_workers=1, shuffle = True)

    testset = torchvision.datasets.ImageFolder(root= 'test',
                                            transform=transform)
    
    test_loader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                             num_workers=1, shuffle = True)

    return train_loader, classes#, val_loader, test_loader, classes

In [0]:
from google.colab import drive
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [0]:
cd /gdrive/My Drive/Colab Notebooks

/gdrive/My Drive/Colab Notebooks


In [0]:
!ls

 Alexnet_resnet_Food_Classification_withgpu.ipynb
'Copy of Alexnet_resnet_Food_Classification_withgpu.ipynb'
'Copy of Copy of Food_Classification_withgpu.ipynb.gdoc'
 Food_Classification.ipynb
 Hyperparameters.gsheet
 model_ResNet_bs32_lr0.0001_epoch0
 model_ResNet_bs32_lr0.0001_epoch1
 model_ResNet_bs32_lr0.0001_epoch10
 model_ResNet_bs32_lr0.0001_epoch11
 model_ResNet_bs32_lr0.0001_epoch12
 model_ResNet_bs32_lr0.0001_epoch13
 model_ResNet_bs32_lr0.0001_epoch14
 model_ResNet_bs32_lr0.0001_epoch15
 model_ResNet_bs32_lr0.0001_epoch16
 model_ResNet_bs32_lr0.0001_epoch17
 model_ResNet_bs32_lr0.0001_epoch18
 model_ResNet_bs32_lr0.0001_epoch19
 model_ResNet_bs32_lr0.0001_epoch19_train_err.csv
 model_ResNet_bs32_lr0.0001_epoch19_train_loss.csv
 model_ResNet_bs32_lr0.0001_epoch19_val_err.csv
 model_ResNet_bs32_lr0.0001_epoch19_val_loss.csv
 model_ResNet_bs32_lr0.0001_epoch2
 model_ResNet_bs32_lr0.0001_epoch3
 model_ResNet_bs32_lr0.0001_epoch4
 model_ResNet_bs32_lr0.0001_epoch5
 model_ResNet_b

In [0]:
classes = ('falafel','apple_pie','donuts','french_fries','macarons','nachos','onion_rings','oysters','pizza', 'mussels')

transform = transforms.Compose(
    [transforms.Resize((224,224), interpolation=2), transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])

trainset = torchvision.datasets.ImageFolder(root = 'train',
                                             transform=transform)

train_loader = torch.utils.data.DataLoader(trainset, batch_size = 64,
                                          num_workers = 1, shuffle = True)
val_set = torchvision.datasets.ImageFolder(root='val',
                                                 transform=transform)
val_loader = torch.utils.data.DataLoader(val_set, batch_size=64,
                                              num_workers=1, shuffle = True)

testset = torchvision.datasets.ImageFolder(root= 'test',
                                            transform=transform)
    
test_loader = torch.utils.data.DataLoader(testset, batch_size=64,
                                             num_workers=1, shuffle = True)

In [0]:
trainset.class_to_idx


{'apple_pie': 0,
 'donuts': 1,
 'falafel': 2,
 'french_fries': 3,
 'macarons': 4,
 'mussels': 5,
 'nachos': 6,
 'onion_rings': 7,
 'oysters': 8,
 'pizza': 9}

In [0]:
train_loader_iter = iter(train_loader)
imgs, labels = next(train_loader_iter)
print(imgs.shape)
print(labels)
print(labels.shape)

torch.Size([64, 3, 224, 224])
tensor([3, 6, 4, 5, 0, 1, 1, 0, 8, 2, 7, 6, 9, 8, 5, 0, 2, 4, 2, 7, 2, 3, 0, 9,
        9, 9, 8, 5, 4, 2, 0, 6, 6, 0, 6, 2, 4, 7, 3, 3, 4, 8, 8, 0, 5, 3, 4, 4,
        2, 6, 2, 4, 8, 7, 1, 4, 2, 8, 5, 7, 1, 8, 9, 1])
torch.Size([64])


# 2. Convolutional Network - Baseline Model 
Build a convolutional neural network model that takes the (224x224 RGB) image as input.

In [0]:
import torch.nn as nn
import torch.nn.functional as F
 
class LargeNet(nn.Module):
    def __init__(self):
        super(LargeNet, self).__init__()
        self.name = "large"
        self.conv1 = nn.Conv2d(3, 32, 3, padding = 1) 
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(32, 64, 3, padding = 1)
        self.conv3 = nn.Conv2d(64, 128, 3, padding = 1)
        
        self.fc1 = nn.Linear(128 * 28 * 28, 128)
        self.fc2 = nn.Linear(128, 9)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = x.view(-1, 128 * 28 * 28)  
        x = F.relu(self.fc1(x))
        x = self.fc2(x) 
        x = x.squeeze(1) # Flatten to [batch_size]
        return x

# 3. Training 
Train your network. Plot the training curve.

Make sure that you are checkpointing frequently!

In [0]:
import torch.nn as nn
import torch.nn.functional as F
# convolutional neural network, 
class LargeNet(nn.Module):
    def __init__(self):
        super(LargeNet, self).__init__()
        self.name = "large"
        self.conv1 = nn.Conv2d(3, 5, 3, padding = 1) 
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(5, 10, 3, padding = 1)
        self.conv3 = nn.Conv2d(10, 15, 3, padding = 1)
        self.fc1 = nn.Linear(15 * 28 * 28 , 64)
        self.fc2 = nn.Linear(64, 10)

    def forward(self, x):
        
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        
        x = x.view(-1, 15 * 28 * 28)  
        x = F.relu(self.fc1(x))
        x = self.fc2(x) 
        x = x.squeeze(1) # Flatten to [batch_size]
        return x

In [0]:
def get_model_name(name, batch_size, learning_rate, epoch):
    """ Generate a name for the model consisting of all the hyperparameter values

    Args:
        config: Configuration object containing the hyperparameters
    Returns:
        path: A string with the hyperparameter name and value concatenated
    """
    path = "model_{0}_bs{1}_lr{2}_epoch{3}".format(name,
                                                   batch_size,
                                                   learning_rate,
                                                   epoch)
    return path


In [0]:
train_loader_iter = iter(train_loader)
imgs, labels = next(train_loader_iter)
print(imgs.shape)
print(labels)
print(labels.shape)

torch.Size([64, 3, 224, 224])
tensor([0, 0, 1, 6, 9, 5, 4, 5, 1, 9, 9, 2, 8, 2, 4, 9, 1, 3, 5, 8, 1, 3, 3, 9,
        4, 0, 9, 3, 5, 8, 4, 3, 8, 7, 7, 8, 5, 6, 3, 8, 5, 7, 3, 9, 4, 7, 5, 7,
        4, 3, 8, 2, 4, 1, 2, 7, 5, 4, 8, 0, 3, 2, 9, 1])
torch.Size([64])


# Training - Baseline Model

In [0]:
def evaluate(net, loader, criterion):
    """ Evaluate the network on the validation set.

     Args:
         net: PyTorch neural network object
         loader: PyTorch data loader for the validation set
         criterion: The loss function
     Returns:
         err: A scalar for the avg classification error over the validation set
         loss: A scalar for the average loss function over the validation set
     """
    total_loss = 0.0
    total_err = 0.0
    total_epoch = 0

    for i, data in enumerate(loader, 0):
        inputs, labels = data
        inputs = inputs.to(device)
        labels = labels.to(device)
        

        outputs = net(inputs)
        loss = criterion(outputs, labels)
        pred = outputs.max(1, keepdim=True)[1] # get the index of the max log-probability
        total_err += pred.ne(labels.view_as(pred)).sum().item()
        total_loss += loss.item()
        total_epoch += len(labels)

    err = float(total_err) / total_epoch
    loss = float(total_loss) / (i + 1)

    return err, loss

In [0]:
def train(net, batch_size=1, learning_rate=0.0001, num_epochs=30):
    transform = transforms.Compose(
        [transforms.Resize((224,224)), transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
    trainset = torchvision.datasets.ImageFolder(root='train',
                                                 transform=transform)
    train_loader = torch.utils.data.DataLoader(trainset, batch_size=64,
                                          num_workers=1, shuffle = True)
    
    train_loader_iter = iter(train_loader)
    imgs, labels = next(train_loader_iter)
    
    
    torch.manual_seed(1000)

   
  
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate, weight_decay=1e-5)
    
    train_err = np.zeros(num_epochs)
    train_loss = np.zeros(num_epochs)
    val_err = np.zeros(num_epochs)
    val_loss = np.zeros(num_epochs)
    
    
    iters, losses, train_acc, val_acc = [], [], [], []
    start_time = time.time()
    # training
    n=0
    for epoch in range(num_epochs):  # loop over the dataset multiple times 
      
        total_train_loss = 0.0
        total_train_err = 0.0

        total_epoch = 0
        for i, data in enumerate(train_loader, 0):
            
            inputs, labels = data
            inputs = inputs.to(device)
            labels = labels.to(device)
                      
            optimizer.zero_grad()
            outputs = net(inputs)
           
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            iters.append(n)
           
            pred = outputs.max(1, keepdim=True)[1] # get the index of the max log-probability
            total_train_err += pred.ne(labels.view_as(pred)).sum().item()
            total_train_loss += loss.item()
            total_epoch += len(labels)
           
            losses.append(float(loss)/batch_size)  
            n += 1# compute *average* loss
        
        train_err[epoch] = float(total_train_err) / total_epoch
     
        train_loss[epoch] = float(total_train_loss) / (i+1)
        
        val_err[epoch], val_loss[epoch] = evaluate(net, val_loader, criterion)
    
        print(("Epoch {}: Train err: {}, Train loss: {}  |" +
               "Validation err: {} , Validation loss:{} ").format(
                   epoch + 1,
            
                   train_err[epoch],
                   train_loss[epoch],
                   val_err[epoch],
                   val_loss[epoch]
                        ))

        # Save the current model (checkpoint) to a file
        model_path = get_model_name(net.name, batch_size, learning_rate, epoch)
        torch.save(net.state_dict(), model_path)

    print('Finished Training')
    end_time = time.time()
    elapsed_time = end_time - start_time
    print("Total time elapsed: {:.2f} seconds".format(elapsed_time))

    # Write the train/test loss/err into CSV file for plotting later
    epochs = np.arange(1, num_epochs + 1)

    np.savetxt("{}_train_err.csv".format(model_path), train_err)
    np.savetxt("{}_train_loss.csv".format(model_path), train_loss)
    np.savetxt("{}_val_err.csv".format(model_path), val_err)
    np.savetxt("{}_val_loss.csv".format(model_path), val_loss)
        
          

In [0]:
# Training Curve
import matplotlib.pyplot as plt
def plot_training_curve(path):
    """ Plots the training curve for a model run, given the csv files
    containing the train/validation error/loss.

    Args:
        path: The base path of the csv files produced during training
    """

    train_err = np.loadtxt("{}_train_err.csv".format(path))
    val_err = np.loadtxt("{}_val_err.csv".format(path))
    train_loss = np.loadtxt("{}_train_loss.csv".format(path))
    val_loss = np.loadtxt("{}_val_loss.csv".format(path))
    plt.title("Train vs Validation Error")
    plt.plot(range(1,16), train_err, label="Train")
    plt.plot(range(1,16), val_err, label="Validation")
    plt.xlabel("Epoch")
    plt.ylabel("Error")
    plt.legend(loc='best')
    plt.show()
    plt.title("Train vs Validation Loss")
    plt.plot(range(1,16), train_loss, label="Train")
    plt.plot(range(1,16), val_loss, label="Validation")
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.legend(loc='best')
    plt.show()

# Hyperparameter Seach - Baseline Model

In [0]:
ne3 = LargeNet()
ne3.to(device)

In [0]:
train(ne3, batch_size = 64, learning_rate=0.01, num_epochs = 15)

Epoch 1: Train err: 0.8581666666666666, Train loss: 2.292994164405985  |Validation err: 0.8265 , Validation loss:2.2168687283992767 
Epoch 2: Train err: 0.8133333333333334, Train loss: 2.1898040238847125  |Validation err: 0.8125 , Validation loss:2.1898394525051117 
Epoch 3: Train err: 0.7908333333333334, Train loss: 2.150301992893219  |Validation err: 0.8325 , Validation loss:2.231725499033928 
Epoch 4: Train err: 0.7218333333333333, Train loss: 2.024061899235908  |Validation err: 0.7735 , Validation loss:2.1689374707639217 
Epoch 5: Train err: 0.6418333333333334, Train loss: 1.8138254690677562  |Validation err: 0.7865 , Validation loss:2.2966490015387535 
Epoch 6: Train err: 0.5516666666666666, Train loss: 1.5930405474723655  |Validation err: 0.806 , Validation loss:2.587955057621002 


KeyboardInterrupt: 

In [0]:
ne3 = LargeNet()
ne3.to(device)
train(ne3, batch_size = 64, learning_rate=0.1, num_epochs = 15)

In [0]:
ne3 = LargeNet()
ne3.to(device)
train(ne3, batch_size = 32, learning_rate=0.001, num_epochs = 15)

In [0]:
ne3 = LargeNet()
ne3.to(device)
train(ne3, batch_size = 32, learning_rate=0.01, num_epochs = 15)

In [0]:
ne3 = LargeNet()
ne3.to(device)
train(ne3, batch_size = 32, learning_rate=0.1, num_epochs = 15)

In [0]:
path = 'model_large_bs64_lr0.001_epoch14'
plot_training_curve(path)


# Test Classification Error

In [0]:
net = LargeNet()
net.to(device)
model_path = get_model_name(net.name, batch_size=64, learning_rate=0.001, epoch=5)
state = torch.load(model_path)
net.load_state_dict(state)

evaluate(net, test_loader, nn.CrossEntropyLoss())


(0.581625, 1.6759631481170654)

In [0]:
The test classification error was about 0.37.

SyntaxError: invalid syntax (<ipython-input-42-3ddd3359581b>, line 1)

In [0]:
len(testset)

8000

# Test Accuracy

In [0]:
def get_accuracy(model):
    data = testset
    total = 0
    correct = 0
    for i, data in enumerate(test_loader, 0):
        inputs, labels = data
        inputs = inputs.to(device)
        labels = labels.to(device)
        
        outputs = net(inputs)
        # We don't need to run F.softmax
        pred = output.max(1, keepdim=True)[1] # get the index of the max log-probability
        correct += pred.eq(labels.view_as(pred)).sum().item()
        total += imgs.shape[0]
    return correct / total

In [0]:
net = LargeNet()
net.to(device)
model_path = get_model_name(net.name, batch_size=64, learning_rate=0.001, epoch=5)
state = torch.load(model_path)
net.load_state_dict(state)




In [0]:
get_accuracy(net)

TypeError: conv2d(): argument 'input' (position 1) must be Tensor, not int

Test accuracy was about 62 %.

# 5. Transfer Learning [16 pt]
For many image classification tasks, it is generally not a good idea to train a very large Deep Neural Network model from scratch due to the enormous compute requirements and lack of sufficient amounts of training data. One of the better option is to try using an existing model that performs a similar task to the one you need to solve. This method of utilizing a pre-trained network for other similar tasks is broadly termed â€œTransfer Learningâ€. In this assignment, we will use Transfer Learning to extract features from the hand gesture images. Then, train a smaller network to use these features as input and classify the hand gestures.

As you have learned from the CNN lecture, Convolution layers extract various features from the images which get utilized by the fully connected layers for correct classification. AlexNet architecture played a pivotal role in establishing Deep Neural Nets as a go-to tool for Image classification problems and we will use an imagenet pre-trained AlexNet model to extract features in this assignment.

In [0]:
from torchvision import transforms

# Set up a transform that scales and crops an image so it has the dimensions
# of the input layer of alexnet
scale_crop = transforms.Compose([
   transforms.Resize(256),
   transforms.CenterCrop(224)
])

# The normalization that was applied to the data when alexnet was trained
normalize = transforms.Normalize(
   mean=[0.485, 0.456, 0.406],
   std=[0.229, 0.224, 0.225]
)

# Put scaling, cropping, normalzing and converting from PIL image to pytorch into one package
preprocess = transforms.Compose([
   scale_crop,
   transforms.ToTensor(),
   normalize
])

In [0]:
trainset = torchvision.datasets.ImageFolder(root = 'train',
                                             transform=preprocess)

train_loader = torch.utils.data.DataLoader(trainset, batch_size = 32,
                                          num_workers = 1, shuffle = True)
val_set = torchvision.datasets.ImageFolder(root='val',
                                                 transform=preprocess)
val_loader = torch.utils.data.DataLoader(val_set, batch_size=32,
                                              num_workers=1, shuffle = True)

testset = torchvision.datasets.ImageFolder(root= 'test',
                                           transform=transform)
    
test_loader = torch.utils.data.DataLoader(testset, batch_size=32,
                                             num_workers=1, shuffle = True)

In [0]:
# Importing relevant Libraries
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import time
import os
import torchvision
import matplotlib.pyplot as plt

from torchvision import datasets, models, transforms
from torch.nn import functional as F
import copy
from torch.autograd import Variable


###############################################################################
# Feature Extraction using AlexNet pretrained model

class AlexNet(nn.Module):
    '''
    Class that loads AlexNet Feature Model ('Convolution layers') with imagenet trained weights
    
    input : image tensors with dimension Lx3x224x224
    
    output : feature tensor with dimension Lx256x6x6
    
    *L - Batch size
    
    '''
    
    def load_weights(self):
        an_builtin = torchvision.models.alexnet(pretrained=True) # Loads the pretrained model weights
        
        features_weight_i = [0, 3, 6, 8, 10]
        for i in features_weight_i:
            self.features[i].weight = an_builtin.features[i].weight
            self.features[i].bias = an_builtin.features[i].bias

    def __init__(self):
        super(AlexNet, self).__init__()
        self.name = "AlexNet"
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=11, stride=4, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(64, 192, kernel_size=5, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(192, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(384, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
        )
        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(256 * 6 * 6, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, 10),
        )
        self.load_weights() # Copies the weights to AlexNetFeatures model layers

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), 256 * 6 * 6)
        x = self.classifier(x)
        return x
        


In [0]:
import torchvision.models

alexNet = torchvision.models.alexnet(pretrained=False)
alexNet

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
    (2): ReLU(inplace)
    (3): Dropout(p

In [0]:
import torchvision.models

resNet18 = torchvision.models.resnet18(pretrained=False)
resNet18

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): Co

In [0]:
netalex = AlexNet()
netalex.to(device)

In [0]:
train(netalex, batch_size = 64, learning_rate = 0.00001, num_epochs = 15)

Epoch 1: Train err: 0.5908333333333333, Train loss: 1.8192532404940178  |Validation err: 0.4129353233830846 , Validation loss:1.2098208628594875 
Epoch 2: Train err: 0.336, Train loss: 1.0047019105008308  |Validation err: 0.36517412935323385 , Validation loss:1.1387120466679335 
Epoch 3: Train err: 0.25916666666666666, Train loss: 0.8018852922510593  |Validation err: 0.3402985074626866 , Validation loss:1.1026504393666983 
Epoch 4: Train err: 0.23766666666666666, Train loss: 0.7051939694805348  |Validation err: 0.3353233830845771 , Validation loss:1.1032313834875822 
Epoch 5: Train err: 0.20016666666666666, Train loss: 0.6161762878615805  |Validation err: 0.3407960199004975 , Validation loss:1.0975917130708694 
Epoch 6: Train err: 0.18283333333333332, Train loss: 0.5571522214945327  |Validation err: 0.31144278606965176 , Validation loss:1.0427784509956837 
Epoch 7: Train err: 0.16216666666666665, Train loss: 0.4949508328387078  |Validation err: 0.2900497512437811 , Validation loss:0.98

In [0]:
train(netalex, batch_size = 32, learning_rate = 0.001, num_epochs = 15)

Epoch 1: Train err: 0.8636666666666667, Train loss: 6.696951688604152  |Validation err: 0.8235 , Validation loss:2.198826481425573 
Epoch 2: Train err: 0.7846666666666666, Train loss: 2.0924417465291123  |Validation err: 0.7975 , Validation loss:2.510080871127901 
Epoch 3: Train err: 0.7626666666666667, Train loss: 2.0622327657456094  |Validation err: 0.7625 , Validation loss:2.4051466385523477 
Epoch 4: Train err: 0.751, Train loss: 2.019769406065028  |Validation err: 0.7625 , Validation loss:2.2512343421814935 
Epoch 5: Train err: 0.7235, Train loss: 1.9731543495299968  |Validation err: 0.7465 , Validation loss:2.336838175380041 
Epoch 6: Train err: 0.6973333333333334, Train loss: 1.9247440936717581  |Validation err: 0.7205 , Validation loss:2.2657625448136103 
Epoch 7: Train err: 0.6743333333333333, Train loss: 1.8526354774515679  |Validation err: 0.6925 , Validation loss:2.011690378189087 
Epoch 8: Train err: 0.6598333333333334, Train loss: 1.827751729082554  |Validation err: 0.723

In [0]:
netalex = AlexNet()
netalex.to(device)
train(netalex, batch_size = 16, learning_rate = 0.0001, num_epochs = 20)

Epoch 1: Train err: 0.3865, Train loss: 1.1424561700922378  |Validation err: 0.2865671641791045 , Validation loss:0.9071825702512075 
Epoch 2: Train err: 0.197, Train loss: 0.6006449099550856  |Validation err: 0.3686567164179104 , Validation loss:1.1458514368250257 
Epoch 3: Train err: 0.11216666666666666, Train loss: 0.35121220500862343  |Validation err: 0.30895522388059704 , Validation loss:1.091131821038231 
Epoch 4: Train err: 0.07083333333333333, Train loss: 0.21470273650707083  |Validation err: 0.309452736318408 , Validation loss:1.3678297010206042 
Epoch 5: Train err: 0.0425, Train loss: 0.1261425785958133  |Validation err: 0.31840796019900497 , Validation loss:1.5226205663075523 
Epoch 6: Train err: 0.03866666666666667, Train loss: 0.11121613052772715  |Validation err: 0.32487562189054725 , Validation loss:1.636424587359504 
Epoch 7: Train err: 0.024333333333333332, Train loss: 0.06661355166517674  |Validation err: 0.3238805970149254 , Validation loss:1.791774418618944 
Epoch 8

In [0]:
netalex = AlexNet()
netalex.to(device)
train(netalex, batch_size = 32, learning_rate = 0.0005, num_epochs = 20)

Downloading: "https://download.pytorch.org/models/alexnet-owt-4df8aa71.pth" to /root/.torch/models/alexnet-owt-4df8aa71.pth
244418560it [00:04, 51536996.70it/s]


Epoch 1: Train err: 0.8931666666666667, Train loss: 2.3129352813071393  |Validation err: 0.900497512437811 , Validation loss:2.3032867908477783 
Epoch 2: Train err: 0.878, Train loss: 2.2840839675132263  |Validation err: 0.8766169154228856 , Validation loss:2.2816719819629 
Epoch 3: Train err: 0.8566666666666667, Train loss: 2.2538155393397554  |Validation err: 0.8671641791044776 , Validation loss:2.270226516420879 
Epoch 4: Train err: 0.8348333333333333, Train loss: 2.218134271337631  |Validation err: 0.836318407960199 , Validation loss:2.219892628609188 
Epoch 5: Train err: 0.8061666666666667, Train loss: 2.1489719976770116  |Validation err: 0.8238805970149253 , Validation loss:2.668194231532869 
Epoch 6: Train err: 0.7178333333333333, Train loss: 1.9751741493001898  |Validation err: 0.7079601990049751 , Validation loss:2.018897395285349 
Epoch 7: Train err: 0.601, Train loss: 1.6993810927614252  |Validation err: 0.6975124378109453 , Validation loss:2.4539715827457487 
Epoch 8: Train

In [0]:
netalex = AlexNet()
netalex.to(device)
train(netalex, batch_size = 64, learning_rate = 0.0005, num_epochs = 20)

Epoch 1: Train err: 0.8835, Train loss: 2.320931708559077  |Validation err: 0.8791044776119403 , Validation loss:2.2924960075862826 
Epoch 2: Train err: 0.8613333333333333, Train loss: 2.2638895029717303  |Validation err: 0.8771144278606965 , Validation loss:2.285294078645252 
Epoch 3: Train err: 0.828, Train loss: 2.179639473874518  |Validation err: 0.818407960199005 , Validation loss:2.2222018563558184 
Epoch 4: Train err: 0.7263333333333334, Train loss: 1.9586204531344962  |Validation err: 0.6567164179104478 , Validation loss:1.811238551896716 
Epoch 5: Train err: 0.6023333333333334, Train loss: 1.652762827721048  |Validation err: 0.6343283582089553 , Validation loss:1.8037584414557806 
Epoch 6: Train err: 0.473, Train loss: 1.3626352485190047  |Validation err: 0.508955223880597 , Validation loss:1.6825155871255058 
Epoch 7: Train err: 0.3635, Train loss: 1.0854070890457073  |Validation err: 0.5084577114427861 , Validation loss:1.517351762642936 
Epoch 8: Train err: 0.30366666666666

In [0]:
netalex = AlexNet()
netalex.to(device)
train(netalex, batch_size = 32, learning_rate = 0.00005, num_epochs = 20)

Epoch 1: Train err: 0.4196666666666667, Train loss: 1.2426844055348254  |Validation err: 0.2990049751243781 , Validation loss:0.920150447459448 
Epoch 2: Train err: 0.219, Train loss: 0.6540741188094971  |Validation err: 0.3567164179104478 , Validation loss:1.1191261514784798 
Epoch 3: Train err: 0.156, Train loss: 0.473246028765719  |Validation err: 0.2860696517412935 , Validation loss:0.926016146228427 
Epoch 4: Train err: 0.10583333333333333, Train loss: 0.3268153307602761  |Validation err: 0.3119402985074627 , Validation loss:1.1126495798428853 
Epoch 5: Train err: 0.06833333333333333, Train loss: 0.21071445062122446  |Validation err: 0.31144278606965176 , Validation loss:1.370934506257375 
Epoch 6: Train err: 0.0505, Train loss: 0.15983863476108998  |Validation err: 0.2716417910447761 , Validation loss:1.151049964957767 
Epoch 7: Train err: 0.031, Train loss: 0.09941372552767713  |Validation err: 0.2796019900497512 , Validation loss:1.213928859385233 
Epoch 8: Train err: 0.0281666

In [0]:
netalex = AlexNet()
netalex.to(device)
train(netalex, batch_size = 64, learning_rate = 0.00005, num_epochs = 20)

Epoch 1: Train err: 0.41883333333333334, Train loss: 1.2419434785842896  |Validation err: 0.3034825870646766 , Validation loss:0.9177590764704204 
Epoch 2: Train err: 0.22016666666666668, Train loss: 0.657989907454937  |Validation err: 0.35124378109452736 , Validation loss:1.1080554543979584 
Epoch 3: Train err: 0.15733333333333333, Train loss: 0.47340591885942096  |Validation err: 0.291044776119403 , Validation loss:0.9185986149878729 
Epoch 4: Train err: 0.103, Train loss: 0.32332505919831867  |Validation err: 0.31044776119402984 , Validation loss:1.113845521022403 
Epoch 5: Train err: 0.0685, Train loss: 0.21185640697466565  |Validation err: 0.3054726368159204 , Validation loss:1.3256247895104545 
Epoch 6: Train err: 0.051, Train loss: 0.15742263427757203  |Validation err: 0.2845771144278607 , Validation loss:1.2137668506493644 
Epoch 7: Train err: 0.029833333333333333, Train loss: 0.09628963585388153  |Validation err: 0.2860696517412935 , Validation loss:1.2755623246942247 
Epoch 8

In [0]:
netalex2 = AlexNet()
netalex2.to(device)
train(netalex2, batch_size = 32, learning_rate = 0.00001, num_epochs = 20)

Epoch 1: Train err: 0.5936666666666667, Train loss: 1.809698923471126  |Validation err: 0.41691542288557215 , Validation loss:1.226113047864702 
Epoch 2: Train err: 0.3368333333333333, Train loss: 1.013751449103051  |Validation err: 0.3497512437810945 , Validation loss:1.0828766529522245 
Epoch 3: Train err: 0.26616666666666666, Train loss: 0.8230058829835121  |Validation err: 0.345273631840796 , Validation loss:1.0834999680519104 
Epoch 4: Train err: 0.23733333333333334, Train loss: 0.7154985222410648  |Validation err: 0.3338308457711443 , Validation loss:1.059084074837821 
Epoch 5: Train err: 0.207, Train loss: 0.6225019794829348  |Validation err: 0.3174129353233831 , Validation loss:1.0316127169699896 
Epoch 6: Train err: 0.18166666666666667, Train loss: 0.5609518210304544  |Validation err: 0.3074626865671642 , Validation loss:1.0131385222313896 
Epoch 7: Train err: 0.17233333333333334, Train loss: 0.5021994854224489  |Validation err: 0.29701492537313434 , Validation loss:0.96660196

In [0]:
netalex2 = AlexNet()
netalex2.to(device)
train(netalex2, batch_size = 32, learning_rate = 0.000005, num_epochs = 20)

Epoch 1: Train err: 0.7033333333333334, Train loss: 2.0962643889670676  |Validation err: 0.5358208955223881 , Validation loss:1.6299511988957722 
Epoch 2: Train err: 0.4156666666666667, Train loss: 1.342432473568206  |Validation err: 0.4223880597014925 , Validation loss:1.2120010058085124 
Epoch 3: Train err: 0.33916666666666667, Train loss: 1.0346959842012284  |Validation err: 0.3925373134328358 , Validation loss:1.2089395097323827 
Epoch 4: Train err: 0.29483333333333334, Train loss: 0.9154414857955689  |Validation err: 0.354228855721393 , Validation loss:1.0923599704863534 
Epoch 5: Train err: 0.27, Train loss: 0.8197406967903705  |Validation err: 0.33880597014925373 , Validation loss:1.0811677045292325 
Epoch 6: Train err: 0.25066666666666665, Train loss: 0.767062412931564  |Validation err: 0.3497512437810945 , Validation loss:1.0846690213869488 
Epoch 7: Train err: 0.23166666666666666, Train loss: 0.7002718274263625  |Validation err: 0.336318407960199 , Validation loss:1.087264839

# Test Accuracy-Alex Net
Report the test accuracy of your best model. How does the test accuracy compare to part 4(d)?

In [0]:
def get_accuracy(model):
    data = testset
    total = 0
    correct =0
    for imgs, labels in torch.utils.data.DataLoader(data, batch_size=32):
        features = myfeature_model(imgs)
        output = net(features) # We don't need to run F.softmax
        pred = output.max(1, keepdim=True)[1] # get the index of the max log-probability
        correct += pred.eq(labels.view_as(pred)).sum().item()
        total += imgs.shape[0]
    return correct / total

In [0]:
get_accuracy(net)

# **ResNet18 **

In [0]:
from torchvision import transforms

# Set up a transform that scales and crops an image so it has the dimensions
# of the input layer of alexnet
scale_crop = transforms.Compose([
   transforms.Resize(256),
   transforms.CenterCrop(224)
])

# The normalization that was applied to the data when alexnet was trained
normalize = transforms.Normalize(
   mean=[0.485, 0.456, 0.406],
   std=[0.229, 0.224, 0.225]
)

# Put scaling, cropping, normalzing and converting from PIL image to pytorch into one package
preprocess = transforms.Compose([
   scale_crop,
   transforms.ToTensor(),
   normalize
])

In [0]:
trainset = torchvision.datasets.ImageFolder(root = 'train',
                                             transform=preprocess)

train_loader = torch.utils.data.DataLoader(trainset, batch_size = 32,
                                          num_workers = 1, shuffle = True)
val_set = torchvision.datasets.ImageFolder(root='val',
                                                 transform=preprocess)
val_loader = torch.utils.data.DataLoader(val_set, batch_size=32,
                                              num_workers=1, shuffle = True)

testset = torchvision.datasets.ImageFolder(root= 'test',
                                           transform=transform)
    
test_loader = torch.utils.data.DataLoader(testset, batch_size=32,
                                             num_workers=1, shuffle = True)

In [0]:
import torch.nn as nn
import math
import torch.utils.model_zoo as model_zoo


__all__ = ['ResNet', 'resnet18', 'resnet34', 'resnet50', 'resnet101',
           'resnet152']


model_urls = {
    'resnet18': 'https://download.pytorch.org/models/resnet18-5c106cde.pth',
    'resnet34': 'https://download.pytorch.org/models/resnet34-333f7ec4.pth',
    'resnet50': 'https://download.pytorch.org/models/resnet50-19c8e357.pth',
    'resnet101': 'https://download.pytorch.org/models/resnet101-5d3b4d8f.pth',
    'resnet152': 'https://download.pytorch.org/models/resnet152-b121ed2d.pth',
}


def conv3x3(in_planes, out_planes, stride=1):
    "3x3 convolution with padding"
    return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride,
                     padding=1, bias=False)


class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, inplanes, planes, stride=1, downsample=None):
        super(BasicBlock, self).__init__()
        self.conv1 = conv3x3(inplanes, planes, stride)
        self.bn1 = nn.BatchNorm2d(planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(planes, planes)
        self.bn2 = nn.BatchNorm2d(planes)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        residual = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            residual = self.downsample(x)

        out += residual
        out = self.relu(out)

        return out


class Bottleneck(nn.Module):
    expansion = 4

    def __init__(self, inplanes, planes, stride=1, downsample=None):
        super(Bottleneck, self).__init__()
        self.conv1 = nn.Conv2d(inplanes, planes, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=stride,
                               padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        self.conv3 = nn.Conv2d(planes, planes * 4, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(planes * 4)
        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        residual = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)

        out = self.conv3(out)
        out = self.bn3(out)

        if self.downsample is not None:
            residual = self.downsample(x)

        out += residual
        out = self.relu(out)

        return out


class ResNet(nn.Module):

    def __init__(self, block, layers, num_classes=10):
        self.inplanes = 64
        super(ResNet, self).__init__()
        self.name = "resNet"

        self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3,
                               bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.layer1 = self._make_layer(block, 64, layers[0])
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2)
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2)
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2)
        self.avgpool = nn.AvgPool2d(7)
        self.fc = nn.Linear(512 * block.expansion, num_classes)

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, math.sqrt(2. / n))
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()

    def _make_layer(self, block, planes, blocks, stride=1):
        downsample = None
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                nn.Conv2d(self.inplanes, planes * block.expansion,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(planes * block.expansion),
            )

        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample))
        self.inplanes = planes * block.expansion
        for i in range(1, blocks):
            layers.append(block(self.inplanes, planes))

        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)

        return x



def resnet18(pretrained=False, **kwargs):
    """Constructs a ResNet-18 model.

    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
    """
    model = ResNet(BasicBlock, [2, 2, 2, 2], **kwargs)
    if pretrained:
        model.load_state_dict(model_zoo.load_url(model_urls['resnet18']))
    return model



def resnet34(pretrained=False, **kwargs):
    """Constructs a ResNet-34 model.

    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
    """
    model = ResNet(BasicBlock, [3, 4, 6, 3], **kwargs)
    if pretrained:
        model.load_state_dict(model_zoo.load_url(model_urls['resnet34']))
    return model




def resnet50(pretrained=False, **kwargs):
    """Constructs a ResNet-50 model.

    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
    """
    model = ResNet(Bottleneck, [3, 4, 6, 3], **kwargs)
    if pretrained:
        model.load_state_dict(model_zoo.load_url(model_urls['resnet50']))
    return model




def resnet101(pretrained=False, **kwargs):
    """Constructs a ResNet-101 model.

    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
    """
    model = ResNet(Bottleneck, [3, 4, 23, 3], **kwargs)
    if pretrained:
        model.load_state_dict(model_zoo.load_url(model_urls['resnet101']))
    return model




def resnet152(pretrained=False, **kwargs):
    """Constructs a ResNet-152 model.

    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
    """
    model = ResNet(Bottleneck, [3, 8, 36, 3], **kwargs)
    if pretrained:
        model.load_state_dict(model_zoo.load_url(model_urls['resnet152']))
    return model

In [0]:
res18net = resnet18(num_classes=10)
res18net.to(device)
train(res18net, batch_size = 32, learning_rate = 0.0001, num_epochs = 20)

Epoch 1: Train err: 0.661, Train loss: 1.8924471507681178  |Validation err: 0.5726368159203981 , Validation loss:1.689476768175761 
Epoch 2: Train err: 0.523, Train loss: 1.5235189364311543  |Validation err: 0.5482587064676617 , Validation loss:1.6152919493024311 
Epoch 3: Train err: 0.414, Train loss: 1.2366426790014227  |Validation err: 0.5114427860696518 , Validation loss:1.5224968308494204 
Epoch 4: Train err: 0.2713333333333333, Train loss: 0.8496165484824079  |Validation err: 0.49900497512437814 , Validation loss:1.542760585981702 
Epoch 5: Train err: 0.102, Train loss: 0.40856025478941327  |Validation err: 0.5208955223880597 , Validation loss:1.6961960016735016 
Epoch 6: Train err: 0.0375, Train loss: 0.180067962471475  |Validation err: 0.5129353233830846 , Validation loss:1.7391987906561956 
Epoch 7: Train err: 0.011, Train loss: 0.08696742569829555  |Validation err: 0.5144278606965174 , Validation loss:1.834957066036406 
Epoch 8: Train err: 0.0021666666666666666, Train loss: 0

In [0]:
res18net = resnet18(num_classes=10)
res18net.to(device)
train(res18net, batch_size = 32, learning_rate = 0.00001, num_epochs = 20)

Epoch 1: Train err: 0.8385, Train loss: 2.2299749686362897  |Validation err: 0.7328358208955223 , Validation loss:2.0489529863236444 
Epoch 2: Train err: 0.707, Train loss: 1.978893595807096  |Validation err: 0.6676616915422886 , Validation loss:1.9167451347623552 
Epoch 3: Train err: 0.6308333333333334, Train loss: 1.8412276011832216  |Validation err: 0.6283582089552239 , Validation loss:1.8465531685995678 
Epoch 4: Train err: 0.5855, Train loss: 1.7314548530477158  |Validation err: 0.6014925373134329 , Validation loss:1.7689531227898976 
Epoch 5: Train err: 0.5445, Train loss: 1.626549836168898  |Validation err: 0.5825870646766169 , Validation loss:1.7276400762891013 
Epoch 6: Train err: 0.5155, Train loss: 1.5440139364688954  |Validation err: 0.5761194029850746 , Validation loss:1.6973343046884688 
Epoch 7: Train err: 0.4866666666666667, Train loss: 1.4642740944598585  |Validation err: 0.5671641791044776 , Validation loss:1.6498697277099368 
Epoch 8: Train err: 0.457, Train loss: 1.

In [0]:
res18net = resnet18(num_classes=10)
res18net.to(device)
train(res18net, batch_size = 32, learning_rate = 0.00005, num_epochs = 20)

Epoch 1: Train err: 0.6908333333333333, Train loss: 1.9662996327623408  |Validation err: 0.5990049751243781 , Validation loss:1.7568683662111797 
Epoch 2: Train err: 0.5445, Train loss: 1.61964606097404  |Validation err: 0.5686567164179105 , Validation loss:1.6615479522281222 
Epoch 3: Train err: 0.45916666666666667, Train loss: 1.381170998228357  |Validation err: 0.5308457711442786 , Validation loss:1.5866614920752389 
Epoch 4: Train err: 0.37566666666666665, Train loss: 1.1511684104483177  |Validation err: 0.5268656716417911 , Validation loss:1.5478026677691747 
Epoch 5: Train err: 0.25416666666666665, Train loss: 0.8416564229955065  |Validation err: 0.4925373134328358 , Validation loss:1.5212908396645197 
Epoch 6: Train err: 0.13583333333333333, Train loss: 0.547311969576998  |Validation err: 0.5144278606965174 , Validation loss:1.603037300563994 
Epoch 7: Train err: 0.052833333333333336, Train loss: 0.30130988724054175  |Validation err: 0.5213930348258706 , Validation loss:1.683346

In [0]:
res18net = resnet18(num_classes=10)
res18net.to(device)
train(res18net, batch_size = 16, learning_rate = 0.00005, num_epochs = 20)

Epoch 1: Train err: 0.6883333333333334, Train loss: 1.9665132228364335  |Validation err: 0.5940298507462687 , Validation loss:1.7574835353427463 
Epoch 2: Train err: 0.5445, Train loss: 1.6201823670813378  |Validation err: 0.5706467661691542 , Validation loss:1.6646455602040366 
Epoch 3: Train err: 0.4608333333333333, Train loss: 1.3846958178154967  |Validation err: 0.5333333333333333 , Validation loss:1.579527665698339 
Epoch 4: Train err: 0.37316666666666665, Train loss: 1.1520917707301201  |Validation err: 0.517910447761194 , Validation loss:1.531101735811385 
Epoch 5: Train err: 0.251, Train loss: 0.8413121446650079  |Validation err: 0.4945273631840796 , Validation loss:1.527711444430881 
Epoch 6: Train err: 0.14033333333333334, Train loss: 0.5439721184208038  |Validation err: 0.5199004975124378 , Validation loss:1.619893953913734 
Epoch 7: Train err: 0.0565, Train loss: 0.301407719704699  |Validation err: 0.53681592039801 , Validation loss:1.7141874820467025 
Epoch 8: Train err: 0

In [0]:
res18net = resnet18(num_classes=10)
res18net.to(device)
train(res18net, batch_size = 16, learning_rate = 0.00001, num_epochs = 20)

Epoch 1: Train err: 0.8006666666666666, Train loss: 2.184048390134852  |Validation err: 0.6970149253731344 , Validation loss:2.0182409343265353 
Epoch 2: Train err: 0.6726666666666666, Train loss: 1.9546241646117353  |Validation err: 0.6427860696517413 , Validation loss:1.8825933914335946 
Epoch 3: Train err: 0.6145, Train loss: 1.8165169028525656  |Validation err: 0.6303482587064677 , Validation loss:1.8197147070415436 
Epoch 4: Train err: 0.5763333333333334, Train loss: 1.708015207280504  |Validation err: 0.5965174129353233 , Validation loss:1.7441735721769787 
Epoch 5: Train err: 0.5291666666666667, Train loss: 1.6083150721610862  |Validation err: 0.5741293532338309 , Validation loss:1.7038453079405285 
Epoch 6: Train err: 0.5006666666666667, Train loss: 1.5287060737609863  |Validation err: 0.5502487562189055 , Validation loss:1.6645165606150552 
Epoch 7: Train err: 0.4706666666666667, Train loss: 1.4452538414204374  |Validation err: 0.5597014925373134 , Validation loss:1.6417683627

In [0]:
res18net = resnet18(num_classes=10)
res18net.to(device)
train(res18net, batch_size = 16, learning_rate = 0.0001, num_epochs = 20)

Epoch 1: Train err: 0.6645, Train loss: 1.9102689243377524  |Validation err: 0.5895522388059702 , Validation loss:1.7171572617122106 
Epoch 2: Train err: 0.5263333333333333, Train loss: 1.5602751602517797  |Validation err: 0.563681592039801 , Validation loss:1.6372319005784535 
Epoch 3: Train err: 0.4445, Train loss: 1.3073043455468847  |Validation err: 0.5159203980099503 , Validation loss:1.5392789642016094 
Epoch 4: Train err: 0.32166666666666666, Train loss: 0.9735555934145096  |Validation err: 0.5184079601990049 , Validation loss:1.5942716257912772 
Epoch 5: Train err: 0.15866666666666668, Train loss: 0.5589618502145118  |Validation err: 0.5024875621890548 , Validation loss:1.6181088212936643 
Epoch 6: Train err: 0.0555, Train loss: 0.25397126820493254  |Validation err: 0.5373134328358209 , Validation loss:1.772815401591952 
Epoch 7: Train err: 0.017833333333333333, Train loss: 0.10649809312630207  |Validation err: 0.5174129353233831 , Validation loss:1.8311938586689176 
Epoch 8: T